1. 시작 타임스탬프 찍기
2. 드라이버 호출
3. 서울시 코로나 발생동향 페이지 접속
4. for문으로 page_down 반복해서 맨 아래까지 내린다
5. while반복문 - 페이지 전환용으로 1번 걸기
6. 페이지 파싱한다
7. For 반복문 돌면서 td 항목별로 안에 데이터 수집
8. No +=1 (no 기본값 1)
9. If no > cnt : break 로 for문 깨기
10. If no > cnt : break로 while문 깨기
11. page_no +=1 (기본값은 1)
12. 드라이브에게 link_text로 다음페이지 번호 클릭하게 시키기 
13. If 다음 페이지 번호 없으면 : 
14. If mark == 3 : break로 while문 종료
Elif :  드라이버에게 확진자 현황 1번부터 1만번 까지 버튼.click()
12.5초 대기 
13. 마크 남기기 mark == 3 (기본값 0주기)

1. 파일 닫기 
2. 드라이버 종료 
3. 데이터프레임 생성
4. 데이터프레임에 칼럼 값 추가 
5. 데이터프레임 csv, xlsx로 저장
6. 종료 타임스탬프 찍기 
7. 전체 시간 계산
8. 사용자에게 총 몇 건 수집했는지 알려주기 
전체 총 소요시간도 알려주기 

끝 


In [1]:
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
from selenium.webdriver.common.keys import Keys

In [2]:
print('-'*100)
print('서울특별시 코로나 발생현황 크롤러')
print('='*100)
try : 
    cnt = int(input('1.총 몇 건의 데이터 수집을 원하십니까? : '))
except ValueError : 
    cnt = 999999999999999
f_dir = input('2.파일 저장 경로를 입력하세요 : ')
if f_dir == "" : 
    f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+'서울시 코로나발생 현황')
os.chdir(f_dir+s+'-'+'서울시 코로나발생 현황')

f_name = s+'-'+'서울시_코로나_발생현황'+'.txt'
fc_name = s+'-'+'서울시_코로나_발생현황'+'.csv'
fx_name = s+'-'+'서울시_코로나_발생현황'+'.xlsx'

#시작 타임스탬프 찍기
s_time = time.time()

#드라이버 호출
path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15')
driver = webdriver.Chrome(path, options=options)

#서울시 페이지 접속
driver.get('https://www.seoul.go.kr/coronaV/coronaStatus.do')
time.sleep(5)

----------------------------------------------------------------------------------------------------
서울특별시 코로나 발생현황 크롤러
1.총 몇 건의 데이터 수집을 원하십니까? : 30000
2.파일 저장 경로를 입력하세요 : 


In [3]:
#페이지 맨 아래까지 내리기 
for a in range(10) : 
    driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
    time.sleep(3)

    
no = 1
page_no = 1
f = open(f_name, 'a', encoding='utf-8')
number_list1 = []
p_n_list1 = []
date_list1 = []
place_list1 = []
travel_list1 = []
contact_list1 = []
go_home_list1 = []


#while반복문으로 페이지 전환 1번 걸기 - 1
while True : 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table_content = soup.find_all('tbody')[3].find_all('tr')
    
    for b in table_content : 
        print('-'*100)
        f.write('-'*100+'\n')
        
        number = b.find('th').get_text().replace('\n', "").strip()
        number_list1.append(number)
        print('1.확진자 번호 : %s' %number)
        f.write('1.확진자 번호 : %s' %number+'\n')
        
        td_list = b.find_all('td')
        
        p_n = td_list[0].get_text().replace('\n', "").strip() #환자번호
        p_n_list1.append(p_n)
        print('2.환자번호 : %s' %p_n)
        f.write('2.환자번호 : %s' %p_n+'\n')
        
        date = td_list[1].get_text().replace('\n', "").strip()#확진일
        date_list1.append(date)
        print('3.확진일 : %s' %date)
        f.write('3.확진일 : %s' %date+'\n')
        
        
        place = td_list[2].get_text().replace('\n', "").strip() #거주지
        place_list1.append(place)
        print('4.거주지 : %s'%place)
        f.write('4.거주지 : %s'%place+'\n')
        
        travel = td_list[3].get_text().replace('\n', "").strip() #여행력
        travel_list1.append(travel)
        print('5.여행력 : %s' %travel)
        f.write('5.여행력 : %s'%travel+'\n')
        
        contact = td_list[4].get_text().replace('\n', "").strip() #접촉력
        contact_list1.append(contact)
        print('6.접촉력 : %s' %contact)
        f.write('6.접촉력 : %s' %contact+'\n')
        
        try : 
            go_home = td_list[5].get_text().replace('\n', "").strip() #퇴원현황
            go_home_list1.append(go_home)
            print('7.퇴원현황 : %s' %go_home)
            f.write('7.퇴원현황 : %s' %go_home+'\n')
        except : 
            go_home_list1.append(' ')
            print('7.퇴원현황 : 입원 중')
            f.write('7.퇴원현황 : 입원 중'+'\n')
            
            
        no += 1
        if no > cnt : 
            break
    if no > cnt : 
        break
    page_no += 1
    try : 
        driver.find_element_by_link_text('%d' %page_no).click()
        time.sleep(7)
    except : 
        break



driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div/div[9]/div[2]/div[2]/div[1]/ul/li[2]/button').click()
page_no = 1
time.sleep(5)

#-2
while True : 
    html1 = driver.page_source
    soup1 = BeautifulSoup(html1, 'html.parser')
    table_content1 = soup1.find_all('tbody')[4].find_all('tr')
    
    for b in table_content1 : 
        print('-'*100)
        f.write('-'*100+'\n')
        
        number = b.find('th').get_text().replace('\n', "").strip()
        number_list1.append(number)
        print('1.확진자 번호 : %s' %number)
        f.write('1.확진자 번호 : %s' %number+'\n')
        
        td_list = b.find_all('td')
        
        p_n = td_list[0].get_text().replace('\n', "").strip() #환자번호
        p_n_list1.append(p_n)
        print('2.환자번호 : %s' %p_n)
        f.write('2.환자번호 : %s' %p_n+'\n')
        
        date = td_list[1].get_text().replace('\n', "").strip()#확진일
        date_list1.append(date)
        print('3.확진일 : %s' %date)
        f.write('3.확진일 : %s' %date+'\n')
        
        
        place = td_list[2].get_text().replace('\n', "").strip() #거주지
        place_list1.append(place)
        print('4.거주지 : %s'%place)
        f.write('4.거주지 : %s'%place+'\n')
        
        travel = td_list[3].get_text().replace('\n', "").strip() #여행력
        travel_list1.append(travel)
        print('5.여행력 : %s' %travel)
        f.write('5.여행력 : %s'%travel+'\n')
        
        contact = td_list[4].get_text().replace('\n', "").strip() #접촉력
        contact_list1.append(contact)
        print('6.접촉력 : %s' %contact)
        f.write('6.접촉력 : %s' %contact+'\n')
        
        try : 
            go_home = td_list[5].get_text().replace('\n', "").strip() #퇴원현황
            go_home_list1.append(go_home)
            print('7.퇴원현황 : %s' %go_home)
            f.write('7.퇴원현황 : %s' %go_home+'\n')
        except : 
            go_home_list1.append(' ')
            print('7.퇴원현황 : 입원 중')
            f.write('7.퇴원현황 : 입원 중'+'\n')
            
            
        no += 1
        if no > cnt : 
            break
    if no > cnt : 
        break
    page_no += 1
    try : 
        driver.find_element_by_link_text('%d' %page_no).click()
        time.sleep(7)
    except : 
        break

f.close()
driver.quit()


covid = pd.DataFrame()
covid['확진자 번호'] = number_list1
covid['환자번호'] = p_n_list1
covid['확진일자'] = date_list1
covid['거주지'] = place_list1
covid['여행력'] = travel_list1
covid['접촉력'] = contact_list1
covid['퇴원현황'] = go_home_list1
covid.to_csv(fc_name, encoding='utf-8-sig')
covid.to_excel(fx_name)

e_time = time.time()
total_time = e_time - s_time

print('='*100)
print('요청하신 데이터수집 %s건 중 %s 건을 수집했습니다' %(cnt, (no-1)))
print('데이터 수집에 총 소요된 시간은 %s초 입니다' %round(total_time,2))
print('='*100)

            
        
        
        
        
        
        

----------------------------------------------------------------------------------------------------
1.확진자 번호 : 29281
2.환자번호 : 92960
3.확진일 : 2021-03-08
4.거주지 : 타시도
5.여행력 : -
6.접촉력 : 감염경로 조사중
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 29280
2.환자번호 : 92910
3.확진일 : 2021-03-08
4.거주지 : 타시도
5.여행력 : -
6.접촉력 : 감염경로 조사중
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 29279
2.환자번호 : 93239
3.확진일 : 2021-03-08
4.거주지 : 양천구
5.여행력 : -
6.접촉력 : 기타 확진자 접촉
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 29278
2.환자번호 : 93195
3.확진일 : 2021-03-08
4.거주지 : 은평구
5.여행력 : -
6.접촉력 : 기타 확진자 접촉
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 29277
2.환자번호 : 93236
3.확진일 : 2021-03-08
4.거주지 : 은평구
5.여행력 : -
6.접촉력 : 경기 안성시 축산물공판

----------------------------------------------------------------------------------------------------
1.확진자 번호 : 29181
2.환자번호 : 92856
3.확진일 : 2021-03-08
4.거주지 : 마포구
5.여행력 : -
6.접촉력 : 기타 확진자 접촉
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 29180
2.환자번호 : 92945
3.확진일 : 2021-03-08
4.거주지 : 동대문구
5.여행력 : -
6.접촉력 : 기타 확진자 접촉
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 29179
2.환자번호 : 92684
3.확진일 : 2021-03-07
4.거주지 : 중랑구
5.여행력 : -
6.접촉력 : 강동구 소재 고등학교 관련
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 29178
2.환자번호 : 92531
3.확진일 : 2021-03-07
4.거주지 : 타시도
5.여행력 : -
6.접촉력 : 기타 확진자 접촉
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 29177
2.환자번호 : 92582
3.확진일 : 2021-03-07
4.거주지 : 양천구
5.여행력 : -
6.접촉력 : 감염경로

----------------------------------------------------------------------------------------------------
1.확진자 번호 : 29081
2.환자번호 : 92695
3.확진일 : 2021-03-07
4.거주지 : 성동구
5.여행력 : -
6.접촉력 : 기타 확진자 접촉
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 29080
2.환자번호 : 92508
3.확진일 : 2021-03-05
4.거주지 : 동작구
5.여행력 : -
6.접촉력 : 기타 확진자 접촉
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 29079
2.환자번호 : 92183
3.확진일 : 2021-03-06
4.거주지 : 동작구
5.여행력 : -
6.접촉력 : 기타 확진자 접촉
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 29078
2.환자번호 : 92455
3.확진일 : 2021-03-06
4.거주지 : 광진구
5.여행력 : -
6.접촉력 : 기타 확진자 접촉
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 29077
2.환자번호 : 92303
3.확진일 : 2021-03-06
4.거주지 : 영등포구
5.여행력 : -
6.접촉력 : 기타 확진자 접촉

----------------------------------------------------------------------------------------------------
1.확진자 번호 : 28981
2.환자번호 : 92134
3.확진일 : 2021-03-06
4.거주지 : 동대문구
5.여행력 : -
6.접촉력 : 동대문구 소재 병원 관련('21.2월)
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 28980
2.환자번호 : 92215
3.확진일 : 2021-03-06
4.거주지 : 동작구
5.여행력 : -
6.접촉력 : 기타 확진자 접촉
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 28979
2.환자번호 : 92210
3.확진일 : 2021-03-06
4.거주지 : 동작구
5.여행력 : -
6.접촉력 : 영등포구 소재 음식점 관련
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 28978
2.환자번호 : 92135
3.확진일 : 2021-03-06
4.거주지 : 동대문구
5.여행력 : -
6.접촉력 : 동대문구 소재 병원 관련('21.2월)
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 28977
2.환자번호 : 92208
3.확진일 : 2021-03-06
4.거주지 : 동

----------------------------------------------------------------------------------------------------
1.확진자 번호 : 28881
2.환자번호 : 91771
3.확진일 : 2021-03-05
4.거주지 : 송파구
5.여행력 : -
6.접촉력 : 감염경로 조사중
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 28880
2.환자번호 : 91772
3.확진일 : 2021-03-05
4.거주지 : 송파구
5.여행력 : -
6.접촉력 : 감염경로 조사중
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 28879
2.환자번호 : 91831
3.확진일 : 2021-03-05
4.거주지 : 노원구
5.여행력 : -
6.접촉력 : 기타 확진자 접촉
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 28878
2.환자번호 : 91835
3.확진일 : 2021-03-05
4.거주지 : 노원구
5.여행력 : -
6.접촉력 : 기타 확진자 접촉
7.퇴원현황 : 
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 28877
2.환자번호 : 91852
3.확진일 : 2021-03-05
4.거주지 : 동대문구
5.여행력 : -
6.접촉력 : 동대문구 소재 병원 

...

----------------------------------------------------------------------------------------------------
1.확진자 번호 : 400
2.환자번호 : 9554
3.확진일 : 2020-03-28
4.거주지 : 관악구
5.여행력 : 미국
6.접촉력 : 해외유입
7.퇴원현황 : 퇴원
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 399
2.환자번호 : 9525
3.확진일 : 2020-03-28
4.거주지 : 서초구
5.여행력 : 미국
6.접촉력 : 해외유입
7.퇴원현황 : 퇴원
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 398
2.환자번호 : 9521
3.확진일 : 2020-03-28
4.거주지 : 구로구
5.여행력 : 영국
6.접촉력 : 해외유입
7.퇴원현황 : 퇴원
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 397
2.환자번호 : 9597
3.확진일 : 2020-03-28
4.거주지 : 기타
5.여행력 : -
6.접촉력 : 만민중앙교회
7.퇴원현황 : 퇴원
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 396
2.환자번호 : 9543
3.확진일 : 2020-03-28
4.거주지 : 동작구
5.여행력 : 브라질
6.접촉력 : 해외유입
7.퇴원현황 : 퇴원
--------------

----------------------------------------------------------------------------------------------------
1.확진자 번호 : 300
2.환자번호 : 8782
3.확진일 : 2020-03-20
4.거주지 : 노원구
5.여행력 : -
6.접촉력 : Ace 손해보험
7.퇴원현황 : 퇴원
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 299
2.환자번호 : 8612
3.확진일 : 2020-03-19
4.거주지 : 영등포구
5.여행력 : 필리핀
6.접촉력 : 해외유입
7.퇴원현황 : 퇴원
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 298
2.환자번호 : 8588
3.확진일 : 2020-03-19
4.거주지 : 영등포구
5.여행력 : -
6.접촉력 : 감염경로 조사중
7.퇴원현황 : 퇴원
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 297
2.환자번호 : 8614
3.확진일 : 2020-03-19
4.거주지 : 동작구
5.여행력 : -
6.접촉력 : 감염경로 조사중
7.퇴원현황 : 퇴원
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 296
2.환자번호 : 8603
3.확진일 : 2020-03-19
4.거주지 : 송파구
5.여행력 : -
6.접촉력 : Ace 손해보험
7.퇴원현황 : 퇴원


----------------------------------------------------------------------------------------------------
1.확진자 번호 : 200
2.환자번호 : 7555
3.확진일 : 2020-03-10
4.거주지 : 양천구
5.여행력 : -
6.접촉력 : Ace 손해보험
7.퇴원현황 : 퇴원
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 199
2.환자번호 : 7725
3.확진일 : 2020-03-10
4.거주지 : 양천구
5.여행력 : -
6.접촉력 : Ace 손해보험
7.퇴원현황 : 퇴원
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 198
2.환자번호 : 7539
3.확진일 : 2020-03-10
4.거주지 : 양천구
5.여행력 : -
6.접촉력 : Ace 손해보험
7.퇴원현황 : 퇴원
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 197
2.환자번호 : 7524
3.확진일 : 2020-03-10
4.거주지 : 양천구
5.여행력 : -
6.접촉력 : Ace 손해보험
7.퇴원현황 : 퇴원
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 196
2.환자번호 : 7619
3.확진일 : 2020-03-10
4.거주지 : 관악구
5.여행력 : -
6.접촉력 : Ace 손해보험
7.퇴원현황 : 퇴원


----------------------------------------------------------------------------------------------------
1.확진자 번호 : 100
2.환자번호 : 5377
3.확진일 : 2020-03-04
4.거주지 : 동대문구
5.여행력 : -
6.접촉력 : 타시도 확진자 접촉
7.퇴원현황 : 퇴원
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 99
2.환자번호 : 5592
3.확진일 : 2020-03-04
4.거주지 : 도봉구
5.여행력 : -
6.접촉력 : 동대문관련(교회pc방)
7.퇴원현황 : 퇴원
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 98
2.환자번호 : 4645
3.확진일 : 2020-03-02
4.거주지 : 강남구
5.여행력 : -
6.접촉력 : 감염경로 조사중
7.퇴원현황 : 퇴원
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 97
2.환자번호 : 4517
3.확진일 : 2020-03-02
4.거주지 : 서초구
5.여행력 : -
6.접촉력 : 기타 확진자 접촉
7.퇴원현황 : 퇴원
----------------------------------------------------------------------------------------------------
1.확진자 번호 : 96
2.환자번호 : 4743
3.확진일 : 2020-03-02
4.거주지 : 영등포구
5.여행력 : -
6.접촉력 : 감염경로 조사중
7.퇴원현황 

요청하신 데이터수집 30000건 중 21300 건을 수집했습니다
데이터 수집에 총 소요된 시간은 1570.56초 입니다
